# Data for TRAPPIST-1 system
Source: Nature 2017

In [1]:
import pandas as pd
import yaml
import numpy as np

In [2]:
with open('bodies.yaml', 'r') as f:
    raw_planet_data = yaml.load(f)

In [3]:
earth = {'mass':5.972e24,   # in kg
         'radius': 6.371e6  # in m
        }
sun = {'mass':1.989e30,     # in kg
       'radius': 695.8e6    # in m
      }

conversions = {'meter2au': 1. / 149597870700}

In [4]:
columns = ['name', 'mass', 'period', 'radius', 'semi-major', 'eccentricity']

planet_data = pd.DataFrame(columns=columns)
star_data = pd.DataFrame(columns=columns)

for d in raw_planet_data.keys():
    current_dict = raw_planet_data[d]
    current_dict['name'] = d
    for c in current_dict.keys():
        if current_dict[c] == 'NA':
            current_dict[c] = np.nan
    if d == 'star':
        pop_columns = [current_dict['name']] + \
            [float(x) for x in [current_dict['mass'], np.nan, 
                                current_dict['radius'],np.nan,np.nan]]
        df = pd.DataFrame([pop_columns], columns=columns)
        star_data = star_data.append(df)
        continue
    pop_columns = [current_dict['name']] + \
            [float(x) for x in [current_dict['mass'], current_dict['period'], 
                                 current_dict['radius'], current_dict['semi-major'], 
                                 current_dict['eccentricity']]]        
    df = pd.DataFrame([pop_columns], columns=columns)
    planet_data = planet_data.append(df, ignore_index=True)
planet_data = planet_data.sort_values("semi-major").reset_index(drop=True)

Original planet data are:
- **mass** in earth masses
- **period** in earth days
- **radius** in earth radii
- **semi-major** in $10^{-3}$ au

In [5]:
planet_data

,name,mass,period,radius,semi-major,eccentricity
0,b,0.85,1.510871,1.086,11.11,0.081
1,c,1.38,2.421823,1.056,15.21,0.083
2,d,0.41,4.049610,0.772,21.44,0.070
3,e,0.62,6.099615,0.918,28.17,0.085
4,f,0.68,9.206690,1.045,37.10,0.063
5,g,1.34,12.352940,1.127,45.10,0.061
6,h,NaN,20.000000,0.755,63.00,NaN


Original star data are:
- **mass** in sun masses
- **radius** in sun radii

In [6]:
star_data

,name,mass,period,radius,semi-major,eccentricity
0,star,0.0802,NaN,0.117,NaN,NaN


## Convert to local units

Using TRAPPIST units:
- time: earth days
- distance 1/1000 AU
- mass: TRAPPIST-1 mass

In [7]:
star_data['mass'][0]

0.080199999999999994

In [8]:
mass_star = star_data['mass'][0]  # in solar masses
system = planet_data.copy()
system['mass'] *= earth['mass']/sun['mass']
system = system.append(star_data[star_data['name'] == 'star'], ignore_index=True)
system['mass'] /= mass_star

**Mass** is now in multiples of the star's mass; **radius** is still in earth radii for planets and sun radius for star. **Semi-major** in $10^{-3}$ au. **Period** in earth days.

In [9]:
system

,name,mass,period,radius,semi-major,eccentricity
0,b,0.000032,1.510871,1.086,11.11,0.081
1,c,0.000052,2.421823,1.056,15.21,0.083
2,d,0.000015,4.049610,0.772,21.44,0.070
3,e,0.000023,6.099615,0.918,28.17,0.085
4,f,0.000025,9.206690,1.045,37.10,0.063
5,g,0.000050,12.352940,1.127,45.10,0.061
6,h,NaN,20.000000,0.755,63.00,NaN
7,star,1.000000,NaN,0.117,NaN,NaN


In [10]:
system.to_hdf("trappist_bodies.hdf5", "LOCAL_UNITS")

## Convert to solar system units:

In [11]:
bodies = planet_data.copy()
bodies['mass'] *= earth['mass']/sun['mass'] # in M_s
bodies['radius'] *= earth['radius'] * conversions['meter2au'] # in AU
bodies['semi-major'] *= 1e-3 # in AU
bodies['period'] /= 365.242 # in solar years

star_only = star_data[star_data['name'] == 'star'].copy()
star_only['radius'] *= sun['radius'] * conversions['meter2au'] # in AU
bodies = bodies.append(star_only, ignore_index=True)

**Mass** in solar masses, **period** in years, **radius** in au, **semi-major** in au.

In [12]:
bodies

,name,mass,period,radius,semi-major,eccentricity
0,b,0.000003,0.004137,0.000046,0.01111,0.081
1,c,0.000004,0.006631,0.000045,0.01521,0.083
2,d,0.000001,0.011087,0.000033,0.02144,0.070
3,e,0.000002,0.016700,0.000039,0.02817,0.085
4,f,0.000002,0.025207,0.000045,0.03710,0.063
5,g,0.000004,0.033821,0.000048,0.04510,0.061
6,h,NaN,0.054758,0.000032,0.06300,NaN
7,star,0.080200,NaN,0.000544,NaN,NaN


In [13]:
bodies.to_hdf("trappist_bodies.hdf5", "SOLAR_UNITS")